## Data exploration

In [2]:
# Set table name
table_name = "faam_dataset_v4"

# Create DF from table
tweet_df = sqlContext.table(table_name)

# Random sampling
#tweet_df = tweet_df.sample(False, 0.2)

# Display schema and data sample
tweet_df.printSchema()
tweet_df.show(10)

-- quick facts

In [4]:
# Quick df facts
df_size = tweet_df.count()
num_unique_tweets = tweet_df.select('tweet_id').distinct().count()
num_unique_users = tweet_df.select('user_id').distinct().count()

print("Number of records: {}".format(df_size))
print("Number of unique tweets: {}".format(num_unique_tweets))
print("Number of unique users: {}".format(num_unique_users))

-- User tweet frequency distribution

In [6]:
# User tweet frequency distribution
user_tweet_count_df = tweet_df.groupBy("user_id").count()

# Retrieve max user tweet frequency
max_count = user_tweet_count_df.agg({"count": "max"}).collect()[0][0]


# Iterate through frequency ranges from 0 to max_count
step = 200
for threshold in range(0, max_count, step):
    
    # Determine freq of users with a tweet count in range(threshold, threshold+step)
    step_freq = user_tweet_count_df\
    .filter("count>{}".format(threshold))\
    .filter("count<{}".format(threshold+step))\
    .count()
    
    # Display bar the given tweet count bucket
    print("{threshold}{indent}{freqdots}".format(
      threshold=threshold,
      indent=' ' * (5-len(str(threshold))),
      freqdots='.' * step_freq))

### Count of tweets that contain a given keyword

#### Define regex for detecting the keyword

In [8]:
import re

# Define regular expression
substr = 'lol'
regex = "(?<![a-z])({substr})(?![a-z])".format(
  substr=substr)

# Test regular expression
test_values = [
  ("lollipop", False),
  ("lol! ", True),
  ("...lol", True),
  ("lmao lol haha", True),
  ("/LoL/", True)
  ]

test_results = []
for test_val in test_values:
    test_result = (re.search(regex, test_val[0].lower()) is not None) ==  test_val[1]
    test_results.append(test_result)

if all(test_results):
    print("SUCCESS: the regex passed ALL the tests")
else:
    failed_test = ', '.join([str(test_index+1) for test_index, test_res in list(enumerate(test_results)) if not test_res])
    print("FAILED test# {}".format(failed_test))

#### Option1: Use an SQL query

In [10]:
# Register the DataFrame as a SQL temporary view
tweet_df.createOrReplaceTempView("tweets")

# Query temp view using SQL syntax
sql("SELECT * FROM tweets WHERE LOWER(text) RLIKE '{}'".format(regex)).count()

#### Option#2: Use RDD `filter()`

In [12]:
import re

(
tweet_df
    .select("text")
    .rdd
    .map(lambda x: unicode(x[0])) # convert row type to string type
    .filter(lambda x: re.search(regex, x.lower()) is not None) # filter records using a custom lambda function
    .count()
)